# Geth Docker Setup
Not done yet lol!!!

In [3]:
import os

path = os.path.abspath("")
print(path)

store_path = os.path.join(path, "geth/storage")
os.makedirs(store_path, exist_ok=True)

print(store_path)

d:\Project\swinburne\inno\backend_project\demo2\scripts
d:\Project\swinburne\inno\backend_project\demo2\scripts\geth/storage


In [8]:
# Command: docker run -it -p 0.0.0.0:8545:8545 -v C:\geth:/home ethereum/client-go:alltools-stable
result = os.system(f"docker run -it -p 0.0.0.0:8545:8545 -v {store_path}:/home ethereum/client-go:alltools-stable")
if result != 0:
    print("Error starting docker container")
    exit(1)

Error starting docker container
